In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/token-data/tokinized_train.csv
/kaggle/input/salary/vprod_train/TRAIN_RES_2.csv
/kaggle/input/salary/vprod_train/TRAIN_SAL.csv
/kaggle/input/salary/vprod_train/TRAIN_RES_3.csv
/kaggle/input/salary/vprod_train/TRAIN_RES_5.csv
/kaggle/input/salary/vprod_train/TRAIN_RES_4.csv
/kaggle/input/salary/vprod_train/TRAIN_RES_1.csv
/kaggle/input/salary/vprod_train/VPROD_pipeline.ipynb
/kaggle/input/salary/vprod_train/JOB_LIST.csv
/kaggle/input/salary/vprod_train/README.docx


Загрузим все датасеты с резюме по порядку

In [3]:
train_res_1 = pd.read_csv('/kaggle/input/salary/vprod_train/TRAIN_RES_1.csv')
train_res_2 = pd.read_csv('/kaggle/input/salary/vprod_train/TRAIN_RES_2.csv')
train_res_3 = pd.read_csv('/kaggle/input/salary/vprod_train/TRAIN_RES_3.csv')
train_res_4 = pd.read_csv('/kaggle/input/salary/vprod_train/TRAIN_RES_4.csv')
train_res_5 = pd.read_csv('/kaggle/input/salary/vprod_train/TRAIN_RES_5.csv')

Склеим загруженные датасеты по вертикали

In [4]:
train_res = pd.concat([train_res_1, train_res_2, train_res_3, train_res_4, train_res_5], ignore_index=True)

In [5]:
train_res.head()

,achievements,achievements_modified,company_name,demands,id_cv,job_title
0,NaN,NaN,Управление социальной защиты населения Сердобс...,"Работала,непосредственно, с федеральными льгот...",52860148-26da-11e8-a184-9122a281f90e,инспектор
1,NaN,NaN,Администрация города Сердобска,Работа с реестром муниципальной собственности,52860148-26da-11e8-a184-9122a281f90e,инспектор
2,"Работы не закончены, т.к. нет настоящего желан...","Работы не закончены, т.к. нет настоящего желан...",январь 2016-настоящее времяООО «АРТЕЛЬ СТАРАТЕ...,Курирую работы по рудному направлению: соавтор...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог
3,NaN,NaN,"ОАО «Васильевский рудник», п.Мотыгино, Красноя...",ГРП. Документация керна разведочных скважин,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог
4,NaN,NaN,"ООО «Артель старателей «ТАЛ»», Усть-Нера, Респ...",Поисково-разведочные работы на месторождении р...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог


Взглянем на размер датасета

In [6]:
len(train_res)

9287465

Количество классов в датасете

In [12]:
len(train_res.job_title.unique())

1913

Поскольку датасеты склеивали,в них могут быть повторяющиеся части, оьбязательно нужно это проверить

In [7]:
train_res = train_res.drop_duplicates()

In [8]:
len(train_res)

6323850

Как видим, в комбинированном датасете было около 3 млн повторяющихся записей

Поскольку данных и так очень много, а записи без текстового описания обязанностей малополезны для обучения, можем их просто удалить

In [9]:
train_res = train_res.dropna(subset=['demands'])

In [10]:
len(train_res)

6262250

Теперь поделим набор данных на тренировачную и валидационную выборки, для этого сначала отделим строки датасета, содержащие профессии, которые встречаются только один раз - они дожны участвовать в обучении, а оставшуюся часть поделить в соотношении 80 / 20

In [11]:
single_occurrences = train_res['job_title'].value_counts()[train_res['job_title'].value_counts() == 1].index

removed_rows = train_res[train_res['job_title'].isin(single_occurrences)]
new_train_res = train_res[~train_res['job_title'].isin(single_occurrences)]

In [7]:
len(removed_rows)

185

In [8]:
len(new_train_res.job_title.unique())

1728

In [16]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(new_train_res, test_size=0.2, random_state=42, stratify=new_train_res.job_title)

In [17]:
train_data = pd.concat([train_set, removed_rows], ignore_index=True)

Поскольку не хватает оперативной памяти для обучения на всех данных, придется уменьшать датасет, но нужно сохранить в нем все классы, а поэтому удалять строки только с профессиями, которых в датасете и так много 

In [18]:
class_counts = train_data['job_title'].value_counts()

# Определение лимита для каждого класса
limit_per_class = 2000000 // len(class_counts) 

# Функция для отбора записей
def select_rows(group):
    if len(group) > limit_per_class:
        return group.sample(limit_per_class)
    else:
        return group

# Применение функции к каждой группе
reduced_data = train_data.groupby('job_title').apply(select_rows).reset_index(drop=True)


/tmp/ipykernel_36/2384322063.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reduced_data = train_data.groupby('job_title').apply(select_rows).reset_index(drop=True)


In [19]:
reduced_data

,achievements,achievements_modified,company_name,demands,id_cv,job_title
0,NaN,NaN,"ООО ""Дорожно транспортная помощь""",помощь в оформлении ДТП,586bd0a0-80b8-11ea-8252-69632329477c,аварийный комиссар
1,NaN,NaN,Центр оформления ДТП,"Оформление ДТП, оценка ущерба",af965d40-7f5b-11ea-a964-ab5d2eb93a75,аварийный комиссар
2,NaN,NaN,"ООО ""ДТП-Сервис"" Пермь","проводить осмотр места ДТП, устанавливать хара...",b64f7fd0-806f-11ea-8d41-b905beff6f7a,аварийный комиссар
3,NaN,NaN,"ООО ""ПЕРВАЯ-ОЦЕНОЧНАЯ""",В рамках трудовой функции и должностной инстру...,6d0a11e0-85b6-11ea-9ef7-f3e0740405b3,аварийный комиссар
4,NaN,NaN,компания GoodExpert,"оформление ДТП, осмотр поврежденного транспорт...",a87afb10-9d3c-11e7-bbbf-037acc02728d,аварийный комиссар
...,...,...,...,...,...,...
405259,Многократно награжден грамотами за организацию...,Многократно награжден грамотами за организацию...,Общественная организация Всероссийское обществ...,-проводил первичную проверку правоустанавливаю...,6eeec590-a240-11ea-9eef-69632329477c,юристконсульт
405260,NaN,NaN,"АО ""Ресурс""",Подгонка резисторов под требуемый номинал,737a1390-3854-11ea-8465-736ab11edb0c,юстировщик
405261,NaN,NaN,Ацсм ик,Ремонт средств измерения,0e9780d0-865a-11ea-b536-1fdc17069750,юстировщик
405262,NaN,NaN,"Новгородское производственное объединение №Волна""",Корректировка деталей,8be35ff0-d32f-11eb-926b-c92c5fcfa4a0,юстировщик


**Выбор моделей**

TF-IDF (Term Frequency-Inverse Document Frequency) — это метод, который позволяет извлекать значимые признаки из текстовых данных, таких как описания обязанностей в резюме. Вот почему TF-IDF может быть полезен:
- Учет важности слов: TF-IDF помогает выделить важные слова в документе, уменьшая вес часто встречающихся, но менее информативных слов. Это позволяет сосредоточиться на тех терминах, которые действительно могут различать профессии.
- Простота и эффективность: TF-IDF легко реализуется и быстро вычисляется, что делает его подходящим для обработки больших объемов текстовых данных.
- Совместимость с линейными моделями: TF-IDF преобразует текст в числовые векторы, которые хорошо подходят для использования с линейными моделями, такими как логистическая регрессия. Это позволяет эффективно обучать модель на текстовых данных.

Логистическая регрессия является популярным методом для задач многоклассовой классификации благодаря своей простоте и эффективности. Вот несколько причин, почему она может быть подходящим выбором для задачи классификации профессий по резюме:
- Интерпретируемость: Логистическая регрессия предоставляет коэффициенты, которые можно интерпретировать как влияние каждого признака на вероятность принадлежности к определенному классу. Это может быть полезно для понимания, какие аспекты резюме наиболее значимы для определения профессии.
- Эффективность: Логистическая регрессия хорошо масштабируется на больших наборах данных, что важно при работе с большим количеством классов (1913 в вашем случае).
- Многоклассовая классификация: Для многоклассовой задачи можно использовать подходы, такие как "один против всех" (one-vs-all) или "один против одного" (one-vs-one), которые позволяют применять логистическую регрессию для классификации более чем двух классов.

In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


vectorizer = TfidfVectorizer(max_features=10000)
X_train = vectorizer.fit_transform(reduced_data['demands'])
X_test = vectorizer.transform(test_set['demands'])

y_train = reduced_data['job_title']
y_test = test_set['job_title']

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [14]:
import joblib

#Серриализация моделей
joblib.dump(model, 'logistic_regression_model.pkl')

joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

In [23]:
xt = X_test[:100000]
yt = y_test[:100000]

Функция для получения прогноза по обученной модели

In [32]:
import joblib
import pandas as pd
from sklearn.metrics import f1_score

def evaluate_model(test_set):
    # Загрузка обученной модели и векторизатора
    model = joblib.load('logistic_regression_model.pkl')
    vectorizer = joblib.load('tfidf_vectorizer.pkl')

    # Преобразование тестовых данных
    X_test = vectorizer.transform(test_set['demands'])

    # Получение истинных меток
    y_test = test_set['job_title']

    # Получение предсказаний
    predictions = model.predict(X_test)

    # Расчет F1-меры
    f1 = f1_score(y_test, predictions, average='weighted')

    print(f'F1 Score: {f1:.3f}')

    # Добавление предсказанных меток в тестовый набор
    test_set['predicted_job_title'] = predictions

df = test_set[:100000]
evaluate_model(df)
df

F1 Score: 0.360


/tmp/ipykernel_191/3191654666.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['predicted_job_title'] = predictions


,achievements,achievements_modified,company_name,demands,id_cv,job_title,predicted_job_title
3185919,NaN,NaN,МДОУ детский сад №111,"Кормление детей, уборка группы",de2bc650-859a-11ea-9e83-1fdc17069750,воспитатель,помощник воспитателя
7851660,NaN,NaN,МЧ филиал РЖД,Руководство предприятием,0310e580-f68e-11eb-8efe-ab5d2eb93a75,инженер,директор
7452586,NaN,NaN,ООО Ярпромхозторг,приём и выкладка товара в торговом зале| консу...,f428d250-60c2-11eb-ad09-b905beff6f7a,менеджер,продавец
8969926,NaN,NaN,ГБДОУ Детский сад,Педагог,1cebb6b6-ee46-11e6-94b6-0f468c90bfa7,руководитель,педагог
3825401,NaN,NaN,ООО «УСЛАД»,Первичная документация,180211e0-941e-11ea-96fe-e37b4be0b9ed,бухгалтер,финансист
...,...,...,...,...,...,...,...
1764452,NaN,NaN,Джанкойское отделение Государственного казна...,"class=""достижение-western"" Обрабатывал в автом...",4c5c86e0-8858-11e5-8d54-833b590698f7,специалист,казначей
1396673,NaN,NaN,"структурное подразделение ОАО ""Сургутнефтегаз""","- руководство бригадой, - контроль выполнения ...",88d82960-1cef-11e9-980c-736ab11edb0c,мастер,мастер
475792,NaN,NaN,"СТО ""Жигули""","Ремонт ДВС, КПП, подвески и ходовой части, куз...",4ebf66f0-5c26-11e9-80af-ef76bd2a03c1,слесарь,автомеханик
6230113,NaN,NaN,"ООО ""Бистро-Пронто",Осуществление бесперебойной работы двух рестор...,f0635a20-2d62-11eb-b08f-69632329477c,администратор,бармен
